In [63]:
import pandas as pd
import numpy as np
from pathlib import Path


In [64]:
# set path
original_data = "CL-2023-cb.csv"

# read the file
original_df = pd.read_csv(original_data, encoding = "CP1252")
original_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0905B18B365C9D20,classic_bike,28/01/2023 18:10.0,28/01/2023 28:52.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member
1,B4F0562B05CB5404,electric_bike,23/01/2023 10:12.0,23/01/2023 18:27.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
2,5ABF032895F5D87E,classic_bike,29/01/2023 27:04.0,29/01/2023 32:38.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735944,-74.030383,40.740802,-74.042521,member
3,E7E1F9C53976D2F9,classic_bike,24/01/2023 35:08.0,24/01/2023 42:13.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735986,-74.030364,40.727596,-74.044247,member
4,323165780CA0734B,classic_bike,21/01/2023 44:09.0,21/01/2023 48:08.0,Hamilton Park,JC009,Manila & 1st,JC082,40.727596,-74.044247,40.721651,-74.042884,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988846,7E862701EE7A6A03,classic_bike,20/12/2023 55:41.0,20/12/2023 00:10.0,Heights Elevator,JC059,Bloomfield St & 15 St,HB203,40.748720,-74.040487,40.754530,-74.026580,casual
988847,DFAF91AB91BE25DB,classic_bike,13/12/2023 36:43.0,13/12/2023 42:31.0,Heights Elevator,JC059,Bloomfield St & 15 St,HB203,40.748767,-74.040470,40.754530,-74.026580,member
988848,AEE7B49E6EDFAE6F,classic_bike,18/12/2023 40:00.0,18/12/2023 45:45.0,6 St & Grand St,HB302,Bloomfield St & 15 St,HB203,40.744291,-74.034404,40.754530,-74.026580,member
988849,A38E98956AD72EFB,classic_bike,2/12/2023 42:51.0,2/12/2023 50:42.0,6 St & Grand St,HB302,Bloomfield St & 15 St,HB203,40.744398,-74.034501,40.754530,-74.026580,member


In [65]:
station_info = {}

# Iterate through the DataFrame rows
for index, row in original_df.iterrows():
    # For each start station
    start_station_id = row['start_station_id']
    start_station_name = row['start_station_name']
    if pd.isna(start_station_id):
        start_station_id = 'unknown'
        start_station_name = 'unknown'
    if start_station_id not in station_info:
        station_info[start_station_id] = {'station_name': start_station_name,
                                          'start_lat': row['start_lat'], 
                                          'start_lng': row['start_lng'], 
                                          'start_count': 1, 
                                          'end_count': 0}
    else:
        station_info[start_station_id]['start_count'] += 1
    
    # For each end station
    end_station_id = row['end_station_id']
    end_station_name = row['end_station_name']
    if pd.isna(end_station_id):
        end_station_id = 'unknown'
        end_station_name = 'unknown'
    if end_station_id not in station_info:
        station_info[end_station_id] = {'station_name': end_station_name,
                                        'start_lat': row['end_lat'], 
                                        'start_lng': row['end_lng'], 
                                        'start_count': 0, 
                                        'end_count': 1}
    else:
        station_info[end_station_id]['end_count'] += 1

# Convert the dictionary to a DataFrame
station_info_df = pd.DataFrame.from_dict(station_info, orient='index').reset_index()
station_info_df.columns = ['station_id', 'station_name', 'start_lat', 'start_lng', 'start_count', 'end_count']

# Calculate the total count for each station
station_info_df['total_count'] = station_info_df['start_count'] + station_info_df['end_count']

# Now, 'station_info_df' contains station IDs and names with their respective start_lat, start_lng, start_count, end_count, and total_count, with NaN values in station_id replaced with "unknown"
station_info_df

,station_id,station_name,start_lat,start_lng,start_count,end_count,total_count
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,27262,27310,54572
1,JC009,Hamilton Park,40.727596,-74.044247,21594,21671,43265
2,HB401,Southwest Park - Jackson St & Observer Hwy,40.737551,-74.041664,12680,12834,25514
3,HB408,Marshall St & 2 St,40.740802,-74.042521,12266,12304,24570
4,JC082,Manila & 1st,40.721651,-74.042884,12747,12970,25717
...,...,...,...,...,...,...,...
568,5938.11,E 16 St & Irving Pl,40.735367,-73.987974,1,1,2
569,4098.06,New York Ave & St Marks Ave,40.726129,-74.034369,1,0,1
570,4143.04,3 Ave & Carroll St,40.677027,-73.986500,1,1,2
571,5445.02,Suffolk St & Stanton St,40.726129,-74.034369,1,1,2


In [66]:
#rename columns to latitude and longditude
station_info_df.rename(columns={'start_lng':'Longitude' , 'start_lat':'Latitude'}, inplace=True)
station_info_df

,station_id,station_name,Latitude,Longitude,start_count,end_count,total_count
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,27262,27310,54572
1,JC009,Hamilton Park,40.727596,-74.044247,21594,21671,43265
2,HB401,Southwest Park - Jackson St & Observer Hwy,40.737551,-74.041664,12680,12834,25514
3,HB408,Marshall St & 2 St,40.740802,-74.042521,12266,12304,24570
4,JC082,Manila & 1st,40.721651,-74.042884,12747,12970,25717
...,...,...,...,...,...,...,...
568,5938.11,E 16 St & Irving Pl,40.735367,-73.987974,1,1,2
569,4098.06,New York Ave & St Marks Ave,40.726129,-74.034369,1,0,1
570,4143.04,3 Ave & Carroll St,40.677027,-73.986500,1,1,2
571,5445.02,Suffolk St & Stanton St,40.726129,-74.034369,1,1,2


In [67]:
station_info_df.to_csv('station_info_2023.csv', index=False)

In [68]:
# Combine 2023 and 2024 station info
new_data_df = pd.read_csv('station_info_2024.csv')

# Concatenate the DataFrames vertically
combined_df = pd.concat([station_info_df, new_data_df], ignore_index=True)

# Save the combined DataFrame as a new CSV file
combined_df.to_csv('combined_station_info.csv', index=False)
combined_df

,station_id,station_name,Latitude,Longitude,start_count,end_count,total_count
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,27262,27310,54572
1,JC009,Hamilton Park,40.727596,-74.044247,21594,21671,43265
2,HB401,Southwest Park - Jackson St & Observer Hwy,40.737551,-74.041664,12680,12834,25514
3,HB408,Marshall St & 2 St,40.740802,-74.042521,12266,12304,24570
4,JC082,Manila & 1st,40.721651,-74.042884,12747,12970,25717
...,...,...,...,...,...,...,...
733,JC108,Bergen Ave & Stegman St,40.706689,-74.086764,67,77,144
734,JC051,Union St,40.718054,-74.083381,153,182,335
735,unknown,unknown,40.740000,-74.030000,7,296,303
736,JC063,Jackson Square,40.711163,-74.078880,196,205,401


In [69]:
#Replace Nan values in columns and clean the data for the main csv file
# List of columns to replace NaN values
columns_to_replace = ['ride_id', 'rideable_type', 'started_at', 'ended_at', 
                      'start_station_name', 'start_station_id', 
                      'end_station_name', 'end_station_id', 'member_casual']

# Replace NaN values with 'unknown' for specified columns
original_df[columns_to_replace] = original_df[columns_to_replace].fillna('unknown')
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988851 entries, 0 to 988850
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             988851 non-null  object 
 1   rideable_type       988851 non-null  object 
 2   started_at          988851 non-null  object 
 3   ended_at            988851 non-null  object 
 4   start_station_name  988851 non-null  object 
 5   start_station_id    988851 non-null  object 
 6   end_station_name    988851 non-null  object 
 7   end_station_id      988851 non-null  object 
 8   start_lat           988851 non-null  float64
 9   start_lng           988851 non-null  float64
 10  end_lat             987897 non-null  float64
 11  end_lng             987897 non-null  float64
 12  member_casual       988851 non-null  object 
dtypes: float64(4), object(9)
memory usage: 98.1+ MB


In [70]:
# Replace Nan values in the lat and long columns with '0'
columns_to_replace_with_zero = ['start_lat', 'start_lng', 'end_lat', 'end_lng']

# Replace NaN values with 0 for specified columns
original_df[columns_to_replace_with_zero] = original_df[columns_to_replace_with_zero].fillna(0)
original_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0905B18B365C9D20,classic_bike,28/01/2023 18:10.0,28/01/2023 28:52.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member
1,B4F0562B05CB5404,electric_bike,23/01/2023 10:12.0,23/01/2023 18:27.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
2,5ABF032895F5D87E,classic_bike,29/01/2023 27:04.0,29/01/2023 32:38.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735944,-74.030383,40.740802,-74.042521,member
3,E7E1F9C53976D2F9,classic_bike,24/01/2023 35:08.0,24/01/2023 42:13.0,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735986,-74.030364,40.727596,-74.044247,member
4,323165780CA0734B,classic_bike,21/01/2023 44:09.0,21/01/2023 48:08.0,Hamilton Park,JC009,Manila & 1st,JC082,40.727596,-74.044247,40.721651,-74.042884,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988846,7E862701EE7A6A03,classic_bike,20/12/2023 55:41.0,20/12/2023 00:10.0,Heights Elevator,JC059,Bloomfield St & 15 St,HB203,40.748720,-74.040487,40.754530,-74.026580,casual
988847,DFAF91AB91BE25DB,classic_bike,13/12/2023 36:43.0,13/12/2023 42:31.0,Heights Elevator,JC059,Bloomfield St & 15 St,HB203,40.748767,-74.040470,40.754530,-74.026580,member
988848,AEE7B49E6EDFAE6F,classic_bike,18/12/2023 40:00.0,18/12/2023 45:45.0,6 St & Grand St,HB302,Bloomfield St & 15 St,HB203,40.744291,-74.034404,40.754530,-74.026580,member
988849,A38E98956AD72EFB,classic_bike,2/12/2023 42:51.0,2/12/2023 50:42.0,6 St & Grand St,HB302,Bloomfield St & 15 St,HB203,40.744398,-74.034501,40.754530,-74.026580,member


In [71]:
original_df.rename(columns={'start_lng':'Longitude' , 'start_lat':'Latitude'}, inplace=True)
original_df.drop(columns=['start_station_name', 'end_station_name', 'ride_id'], inplace=True)
original_df

,rideable_type,started_at,ended_at,start_station_id,end_station_id,Latitude,Longitude,end_lat,end_lng,member_casual
0,classic_bike,28/01/2023 18:10.0,28/01/2023 28:52.0,HB101,JC009,40.735938,-74.030305,40.727596,-74.044247,member
1,electric_bike,23/01/2023 10:12.0,23/01/2023 18:27.0,HB101,HB401,40.735938,-74.030305,40.737551,-74.041664,member
2,classic_bike,29/01/2023 27:04.0,29/01/2023 32:38.0,HB101,HB408,40.735944,-74.030383,40.740802,-74.042521,member
3,classic_bike,24/01/2023 35:08.0,24/01/2023 42:13.0,HB101,JC009,40.735986,-74.030364,40.727596,-74.044247,member
4,classic_bike,21/01/2023 44:09.0,21/01/2023 48:08.0,JC009,JC082,40.727596,-74.044247,40.721651,-74.042884,member
...,...,...,...,...,...,...,...,...,...,...
988846,classic_bike,20/12/2023 55:41.0,20/12/2023 00:10.0,JC059,HB203,40.748720,-74.040487,40.754530,-74.026580,casual
988847,classic_bike,13/12/2023 36:43.0,13/12/2023 42:31.0,JC059,HB203,40.748767,-74.040470,40.754530,-74.026580,member
988848,classic_bike,18/12/2023 40:00.0,18/12/2023 45:45.0,HB302,HB203,40.744291,-74.034404,40.754530,-74.026580,member
988849,classic_bike,2/12/2023 42:51.0,2/12/2023 50:42.0,HB302,HB203,40.744398,-74.034501,40.754530,-74.026580,member


In [72]:
print(original_df.columns)

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_id',
       'end_station_id', 'Latitude', 'Longitude', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')


In [73]:
original_df.to_csv('cleaned_2023_data.csv', index=False)